In [1]:
#Read the network
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import matplotlib.pyplot as plt
import operator
import random
from math import *
from math import sqrt
import pandas as pd
import csv
import os
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import community.community_louvain


####################################### Set path to upload file ##############################################
G=nx.Graph()
os.chdir(r"C:\Users\suman\CVoteRank") #Network Dataset -Community Based\K-Air traffic control
file_name='CVoteRank'
#read a graph in text/CSV format
#G = nx.read_edgelist("hi1.txt", create_using = nx.Graph(), nodetype=int)


####################################### Upload graph as a .CSV file ##########################################
G = nx.Graph()
df = pd.read_csv('CVoteRank.csv', squeeze=True)
G = nx.Graph()
nod = []
for i in range(df.shape[0] - 0):
    nod.append(df['Node1'][i])
    nod.append(df['Node2'][i])
tmp = set(nod)
nod = list(tmp)
G.add_nodes_from(nod)
for i in range(df.shape[0] - 0):
    #G.add_edge(df['Node1'][i], df['Node2'][i], weight=df['Weight'][i])
    G.add_edge(df['Node1'][i], df['Node2'][i])
    
g=G
print(G)
# Average Clustering Coefficient
avg_clustering = nx.average_clustering(G)
print(f"Average Clustering Coefficient: {avg_clustering}")

# Average Degree
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
avg_degree = 2 * num_edges / num_nodes
print(f"Average Degree: {avg_degree}")    
    

############################################## Organize Community ############################################

def flip_nodes_and_communities(dict_nodes_communities):
    # Step 1: initialize communities as keys
    new_dict = {}
    for k, v in dict_nodes_communities.items():
        new_dict[v]=[]
    
    # Step 2: Fill in nodes
    for kk,vv in new_dict.items():
        for k,v in dict_nodes_communities.items():
            if dict_nodes_communities[k] == kk:
                new_dict[kk].append(k)
    
    return new_dict


def orderCommunities(c):
    # We index each community starting from 0 and put them in list keys_partition
    # So, if we have 11 communities, then the indexing list keys_partition = [0,1,2,3,4,5,6,7,8,9,10]
    # Use `j` to move till the length of c and iterate `i` = i+1 to organize the community numbering
    # say it is #20, as index 0 and the second community, say #49 as index 1, etc...
    i = 0
    keys_partition = list()
    for j in c:
        keys_partition.append(i)
        i = i + 1

    # Then for each index i aka for each community, we will create a dictionary partition where i is the key
    partition = dict()
    for i in keys_partition:
        partition[i] = []

    # Now we have to fill the partition dictionary
    i = 0
    for j in c:  # for each community j in c
        for k in c[j]:  # for each node in community c
            partition[i].append(k)  # for partition i (organized), append the nodes in this community
        i = i + 1

    return partition

def communityInfo(c, partition):
    print('Number of partitions: ', len(partition))
    l = list()
    for i in c:  # for all communities i in c
        for j in c[i]:  # for all nodes in communities c[i]
            l.append(j)  # add the nodes to l

    print('Number of nodes in the communities detected: ', len(l))

    s = set(l)  # removes repetitive nodes since set() involves only unique inputs
    print('Number of repetitions: ', len(l) - len(s))


def dict_communities_organized_G(G):
    partition = community.community_louvain.best_partition(G)
    communities=partition
    #print("modularity is",community.community_louvain.modularity(partition, G))
    # Step 1
    dict_communities = flip_nodes_and_communities(communities)
    #print("infomap_communities",infomap_communities)
    #dictionary of keys as community number and values as nodes in that community
    
    # Step 2
    dict_communities_organized = orderCommunities(dict_communities)
    #print("infomap_communities_organized",infomap_communities_organized)
    #Arrange in decending order of infomap_communities
    
    #communityInfo(dict_communities, dict_communities_organized)
    return partition, dict_communities_organized


##############################################################################################################
#graph extractor
def isbelong(list1, list2):
    x = 0
    for i in list1:
        for j in list2:
            if i == j:
                x = 1
    return x

def isexist(x, list1):
    k = 0
    for i in list1:
        if x == i:
            k = 1
    return k

def intra_wo(dict_graph_wo, partition):
    dict_node_partition_wo = dict()
    for i in dict_graph_wo: 
        dict_node_partition_wo[i] = [] 
        for j in partition:  
            for k in partition[j]:
                if i == k:
                    dict_node_partition_wo[i].append(j)

    d_copy = dict_graph_wo.copy()
    dict_g_intra_wo = dict()
    for i in d_copy:
        dict_g_intra_wo[i] = []
        for j in d_copy[i]:
            k = isbelong(dict_node_partition_wo[i], dict_node_partition_wo[j])  
            if k == 1:
                dict_g_intra_wo[i].append(j)

    graph_intra_wo = nx.Graph()
    for i in dict_g_intra_wo:
        graph_intra_wo.add_node(i)

    for i in dict_g_intra_wo:
        for j in dict_g_intra_wo[i]:
            graph_intra_wo.add_edges_from([(i, j)])

    return dict_g_intra_wo, graph_intra_wo


def intra_o(g, partition, lo):

    dict_node_partition = dict ()
    for i in g:  
        dict_node_partition[i] = [] 
        for j in partition: 
            for k in partition[j]:  
                if i == k:  
                    dict_node_partition[i].append(j)


    s = set()
    for i in lo:
        for j in dict_node_partition[i]:
            s.add(j) 
    s_sup = set()
    for i in partition:
        s_sup.add(i)  

    for i in s: 
        s_sup.remove(i) 

 
    g_copy = g.copy()
    for i in s_sup:
        for j in partition[i]:
            g_copy.remove_node(j)


    dict_g_intra_o = dict()
    for i in g_copy:
        dict_g_intra_o[i] =[]
        for j in g[i]:
            k = isbelong(dict_node_partition[i], dict_node_partition[j])  # Do they share any community? If so, append
            if k == 1:
                dict_g_intra_o[i].append(j)


    graph_intra_o = nx.Graph()
    for i in dict_g_intra_o:
        graph_intra_o.add_node(i)

    for i in dict_g_intra_o:
        for j in dict_g_intra_o[i]:
            graph_intra_o.add_edges_from([(i, j)])

    return dict_g_intra_o, graph_intra_o


def inter_wo_o(g, partition):

    dict_node_partition = dict()
    for i in g:
        dict_node_partition[i] = []
        for j in partition:
            for k in partition[j]:
                if i == k:
                    dict_node_partition[i].append(j)

    dict_g_inter = dict()
    for i in g:
        dict_g_inter[i] = []
        for j in g[i]:
            k = isbelong(dict_node_partition[i], dict_node_partition[j])
            if k == 0:
                dict_g_inter[i].append(j)


    graph_inter = nx.Graph()
    for i in dict_g_inter:
        graph_inter.add_node(i)

    for i in dict_g_inter:
        for j in dict_g_inter[i]:
            graph_inter.add_edges_from([(i, j)])

    return dict_g_inter, graph_inter

##############################################################################################################
#method overlapping

def isbelong(l1,l2): # Checks if an item in list 1 exists in list 2
    x=0
    for i in l1:
        for j in l2:
            if i==j:
                x=1

    return x

def list_overlapping(g,communities_flipped):
    l_nodes = g.nodes()
    dict_membership_of_all_nodes=dict() # Prepare the dictionary to contain the membership of all nodes
    for i in l_nodes: # For each node
        dict_membership_of_all_nodes[i]=0
        for j in communities_flipped: # For each community x 
            for k in communities_flipped[j]: # For each node in that community x
                if i==k: # If the node i (starting node) occurs in community x, increment
                    dict_membership_of_all_nodes[i]=dict_membership_of_all_nodes[i]+1
                    
    list_overlapping_nodes=[] # Prepare the list to contain the nodes which only overlap
    for i in dict_membership_of_all_nodes: # For each node
        if dict_membership_of_all_nodes[i]>1: # If the membership is greater than 1, it is an overlapping node
            list_overlapping_nodes.append(i)


            
    dict_membership_of_overlapping_nodes=dict() # Prepare the dictionary to contain the membership of overlapping nodes only
    for i in list_overlapping_nodes: # For each overlapping node
        dict_membership_of_overlapping_nodes[i]=dict_membership_of_all_nodes[i] # Get its membership and put it in the dictionary


    return list_overlapping_nodes, dict_membership_of_all_nodes, dict_membership_of_overlapping_nodes
    
    
def remove_overlapping(g,list_overlapping_nodes,communities_flipped):

    # Copy the dictionary communities_flipped
    communities_flipped_copy=dict()
    for i in communities_flipped:
        communities_flipped_copy[i]=[]
    for i in communities_flipped:
        for j in communities_flipped[i]:
            communities_flipped_copy[i].append (j)
     

    # Get the nodes and their neighbors in a dictionary `dict_graph`
    l_nodes = g.nodes()
    dict_graph = dict()  
    for i in l_nodes:
        dict_graph[i] = [] # nodes in the key and their neighbors in a list
    for i in l_nodes: # For each node
        iteri = g.neighbors(i) # Get the neighbors
        for j in iteri: # Access the neighbors
            dict_graph[i].append(j) # Append them to the list 
            
        
    # Remove the overlapping nodes from dict_graph
    for overlapping_node in list_overlapping_nodes: # For each overlapping node
        for i in dict_graph: # For each node (overlapping or not)
            for j in dict_graph[i]: # For the neighbor of each node
                if j == overlapping_node: # If the neighbor is an overlapping node
                    dict_graph[i].remove(overlapping_node) # Remove it
        dict_graph.pop(overlapping_node) # Remove overlapping node from list when done

    # Remove the overlapping nodes from communities_flipped_copy
    for overlapping_node in list_overlapping_nodes: # For each overlapping node
        for i in communities_flipped_copy: # For each community
            for j in communities_flipped_copy[i]: # For each node in that community
                if j == overlapping_node: # If the node is an overalapping node, remove it
                    communities_flipped_copy[i].remove(overlapping_node)

    # Construct the graph without overlapping nodes
    g_without_ov = nx.Graph()
    for i in dict_graph:
        g_without_ov.add_node(i) # Add the nodes first
    l_edges = []
    for i in dict_graph: # For each node that doesn't overlap
        for j in dict_graph[i]: # For the neighbors
            l_edges.append ((i, j)) # Create a list where they connect
    g_without_ov.add_edges_from(l_edges)

    return g_without_ov,dict_graph,communities_flipped_copy
    
    
def intra_wo(dict_graph_wo,communities_flipped):

    dict_node_communities_flipped = dict() 
    for i in dict_graph_wo: # Get the non-overlapping nodes and the communities they participate in 
        dict_node_communities_flipped[i] = []
        for j in communities_flipped:
            for k in communities_flipped[j]:
                if i == k:
                    dict_node_communities_flipped[i].append(j)
                    
    d_copy=dict_graph_wo.copy() 
    dict_graph_intra_wo=dict() # Get the non-overlapping node and its neighbors which are non overlapping in `dict_graph_intra_wo`
    for i in d_copy:
        dict_graph_intra_wo[i] =[]
        for j in d_copy[i]:
            k=isbelong(dict_node_communities_flipped[i],dict_node_communities_flipped[j])
            if k==1:
                dict_graph_intra_wo[i].append(j)
                
    # Construct dict_graph_intra_wo into graph_intra_wo
    graph_intra_wo = nx.Graph ()
    for i in dict_graph_intra_wo:
        graph_intra_wo.add_node (i)

    for i in dict_graph_intra_wo:
        for j in dict_graph_intra_wo[i]:
            graph_intra_wo.add_edges_from ([(i, j)])
    
    return dict_graph_intra_wo, graph_intra_wo
    
def intra_o(g,communities_flipped,list_overlapping_nodes):

    dict_node_communities_flipped = dict () 
    for i in g: # Get all the nodes and the communities they participate in 
        dict_node_communities_flipped[i] = []
        for j in communities_flipped:
            for k in communities_flipped[j]:
                if i == k:
                    dict_node_communities_flipped[i].append(j)
    g_copy=g.copy()
    dict_graph_intra_o=dict()

    # Find the list of communities to delete
    #print(list_overlapping_nodes)
    #print(dict_node_communities_flipped)

    c=set() # Will contain the communities of the overlapping nodes
    for i in list_overlapping_nodes: # For each overlapping node
        for j in dict_node_communities_flipped[i]: # For each community of the overlapping node
            c.add(j) # Add the community
            
    c_sup=set() # Will contain all of the communities to be later processed to contain only communities with non-overlapps in it
    for i in communities_flipped: # For each community
        c_sup.add(i) # Add the community

    for i in c: # Remove communities which have overlaps in them
        c_sup.remove(i)
        
    #Remove the nodes in the communities with no overlaps --> We want overlapping intra nodes only
    for i in c_sup:
        for j in communities_flipped[i]:
            g_copy.remove_node(j) # May be nothing removed if all communities have overlaps in them

    # Build the dictionary of the overlapping intra 
    for i in g_copy: #g_copy not g
        dict_graph_intra_o[i] =[]
        for j in g[i]:
            k=isbelong(dict_node_communities_flipped[i],dict_node_communities_flipped[j])
            if k==1:
                dict_graph_intra_o[i].append(j)

    # Construct the overlapping intra graph
    graph_intra_o = nx.Graph ()
    for i in dict_graph_intra_o:
        graph_intra_o.add_node(i)

    for i in dict_graph_intra_o:
        for j in dict_graph_intra_o[i]:
            graph_intra_o.add_edges_from ([(i, j)])
            
    return dict_graph_intra_o, graph_intra_o

def inter_wo_o(g,communities_flipped):

    dict_node_communities_flipped = dict ()
    for i in g: # Get all the nodes and the communities they participate in 
        dict_node_communities_flipped[i] = []
        for j in communities_flipped:
            for k in communities_flipped[j]:
                if i == k:
                    dict_node_communities_flipped[i].append(j)

    dict_graph_inter=dict() # dict of overlapping and non-overlapping
    for i in  g:
        dict_graph_inter[i]=[]
        for j in g[i]:
            k=isbelong(dict_node_communities_flipped[i],dict_node_communities_flipped[j])
            if k==0:
                dict_graph_inter[i].append(j)

    graph_inter = nx.Graph() # Graph of inter overlapping + non-overlapping
    for i in dict_graph_inter:
        graph_inter.add_node (i)

    for i in dict_graph_inter:
        for j in dict_graph_inter[i]:
            graph_inter.add_edges_from ([(i, j)])
    return dict_graph_inter, graph_inter


##############################################################################################################
G.remove_edges_from(nx.selfloop_edges(G))
partition, dict_communities_organized =dict_communities_organized_G(G)
communities=partition
g=G

lo, membership_all_nodes, membership_overlapping_nodes = list_overlapping(g, dict_communities_organized)

#dict_graph_wo contain dic={nodeid:[node ids in same/different community]} 
g_wo, dict_graph_wo, partition_wo = remove_overlapping(g, lo, dict_communities_organized)

#g_intra_wo contain dic={nodeid:[node ids in same community]} #All nodes and inner community link
#graph_wo_intra store the connectivity in graph format
g_intra_wo, graph_wo_intra = intra_wo(dict_graph_wo, dict_communities_organized) 

#g_intra_wo contain dic={nodeid:[node ids in different community]} #All nodes and outer community link
#g_inter store the connectivity in graph format
g_inter, graph_inter = inter_wo_o(g, dict_communities_organized)


##############################################################################################################
#For getting original graph
dict_graph = dict()
for i in g:
    dict_graph[i] = []
    for j in g[i]:
        dict_graph[i].append(j)

# print('Raw graph information:')
# print(nx.info(g))

# Convert the different dictionaries into graphs to be for calculation of different centralities
g_intra_wo_converted = nx.Graph(g_intra_wo)  #All nodes and inner community link
g_inter_converted = nx.Graph(g_inter) #All nodes and outer community link
dict_graph_converted = nx.Graph(dict_graph) ##All nodes and all inner/outer community link
##############################################################################################################


Graph with 15 nodes and 24 edges
Average Clustering Coefficient: 0.2977777777777778
Average Degree: 3.2


C:\Users\suman\AppData\Local\Temp\ipykernel_18336\2499503137.py:29: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df = pd.read_csv('CVoteRank.csv', squeeze=True)


In [2]:
#CVoteRAnk: Measure Community based spreading ability

#Global ranking: Done by Shannon Entropy
GCdict={}
GCdicts={}
CDR={}
CDRs={}
import math
for i in G.nodes():        #for each node 
    CD=0
    OC=0
    com=set(communities.values()) 
    for j in com:             #for each community
        nei=list(g.neighbors(i))
        count=0
        prob=1
        for k in nei:
            if communities[k]==j:
                count=count+1
                
        #print(count,len(nei))
        
        prob=count/len(nei)
        if prob!=0:
            CD=CD+(-prob*(math.log(prob,10)))
    CDR[i]=CD
GCdict=CDR


##############################################################################################################
#local: by local betweenness
LCdict = nx.betweenness_centrality(g_intra_wo_converted, normalized = True, endpoints = False) 
##############################################################################################################
#Compute the final ranking by global, local and community ranking
CBFdict={}    
for item in G.nodes():
    CBFdict[item]=math.sqrt((1+GCdict[item])*(1+LCdict[item]))
##############################################################################################################
############################################ save in pickle format ###########################################
import pickle
try:  
    filep = open('CBFdict_p', 'wb')
    pickle.dump(CBFdict, filep)
    filep.close()
    
except:
    print("Something went wrong")

In [3]:
#To know the community partition
partition  

#We get {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 2, 13: 2, 14: 2, 15: 2} for the Toy network
#implies: nodes in one community are:{1, 2, 3, 4, 5, 6}, 
#nodes in another community are {7, 8, 9, 10, 11}, 
#nodes in another community are {12, 13, 14, 15}

{1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 2,
 13: 2,
 14: 2,
 15: 2}

In [4]:
from operator import itemgetter
import pickle
import math
###########################################################################################################
def secondHN(G,node):
    neighbor1=[n for n in G.neighbors(node)]
    #print(node1, "1st neighbors are", neighbor1)
    n2=[]
    for node2 in neighbor1:     
        neighbor2=[m for m in G.neighbors(node2)]
        n2=n2+neighbor2
    #for getting the unique node from list
    node2 = []
    for x in n2:
        if x not in node2:
            if x not in neighbor1:
                node2.append(x) 
            
    node2.remove(node)
    return node2

###########################################################################################################
def CBVote(G, n):   #Proposed
    capability = {}
    Nodes = list(G.nodes())
    totdeg = 0
    for node in Nodes:
        capability[node] = [0, CBFdict[node]]
        totdeg += G.degree(node)
    totdeg /= len(Nodes)
    val = 1 / totdeg
    pbestnode=-1
    selected = []
    for i in range(n):
        for node in Nodes:
            capability[node][0] = 0
            #print("Updated voting ability of of node",i,"is", capability[node][1]) #initialy it is CBFdict, then updated in each iteraction
        bes = 0
        bestnode = -1
        for node in Nodes:
            allnodes = list(G.neighbors(node))
            if pbestnode!=-1: #next iteration, previously selcted node will not perticipate
                if pbestnode in allnodes:
                    allnodes.remove(pbestnode)
                    
            for neig in allnodes:
                if capability[neig][1] > 0:
                    capability[node][0] += capability[neig][1]   
            capability[node][0] = capability[node][1]*(CBFdict[node]+math.sqrt(capability[node][0]))   #
            #print("Computed voting Score of", node, "is", capability[node][0])
            if capability[node][0] > bes and node not in selected:
                bes = capability[node][0]
                bestnode = node
        if bestnode == -1:
            break
        pbestnode=bestnode
        selected.append(bestnode)
        capability[bestnode][1]=0
        #print("Best node is",selected)
        neighbors = list(G.neighbors(bestnode))
        for node in neighbors:
            capability[node][1] = 0.1*capability[node][1]
            #print("updated capability of ",node,"is",capability[node][1])
        node2=secondHN(G,bestnode)
        
        for neig in node2:
            capability[neig][1] = math.sqrt(0.1)*capability[neig][1]
            #print("updated capability of ",neig,"is",capability[neig][1])

        caps = {}
        for i in G.nodes():
            caps[i] = capability[i][0]
    return selected, caps

###########################################################################################################
def SIRModel(g, spreaders, beta, S):
    infected_scale = np.array(np.zeros(50))
    ftc = 0
    while S > 0:
        S = S - 1
        infected = spreaders
        status = {}
        for i in g.nodes():
            status[i] = 0
        for i in infected:
            status[i] = 1
        n = g.number_of_nodes()
        infected_nodes = len(infected)
        recovered_nodes = 0
        time_stamp = 0
        infected_scale[time_stamp] = infected_scale[time_stamp] + (infected_nodes + recovered_nodes) / n
        infected = spreaders
        while len(infected) > 0:
            susceptible_to_infected = []
            time_stamp = time_stamp + 1
            for i in infected:
                susceptible = []
                status[i] = 2
                for neighbor in g.neighbors(i):
                    if status[neighbor] == 0:
                        susceptible.append(neighbor)
                total_susceptible = len(susceptible)
                no_of_susceptible_to_infected = round(beta * total_susceptible)
                while no_of_susceptible_to_infected > 0:
                    random_index = random.randint(0, total_susceptible - 1)
                    if susceptible[random_index] not in susceptible_to_infected:
                        susceptible_to_infected.append(susceptible[random_index])
                        status[susceptible[random_index]] = 1
                        no_of_susceptible_to_infected = no_of_susceptible_to_infected - 1
            infected_nodes = len(susceptible_to_infected)
            recovered_nodes = len(infected)
            ftc = ftc + recovered_nodes / n
            infected_scale[time_stamp] = infected_scale[time_stamp] + (infected_nodes + recovered_nodes) / n
            infected = susceptible_to_infected
    return infected_scale, ftc

#####################################################################################################################
def Find_Spreaders_Updated(g, shorted_list, no_of_spreaders):
    return shorted_list[:no_of_spreaders]


def dict2list(d, k):
    selected = []
    i = 0
    for node in sorted(d.items(), key=itemgetter(1), reverse=True):
        if i < k:
            selected.append(node[0])
            i += 1
    return selected

def dict2list(d, k):
    selected = []
    i = 0
    for node in sorted(d.items(), key=itemgetter(1), reverse=True):
        if i < k:
            selected.append(node[0])
            i += 1
    return selected
                             
######################################################################################################################
nodes = list(G.nodes())
edges = list(G.edges())

edges = list(G.edges())
nodes = list(G.nodes())

print(0.15*(g.number_of_nodes()))

k= int(0.15*(g.number_of_nodes()))
k=5 ##for toy network, we want top 5 spreaders 
S=100
T=50
beta=0.01
#print(k)


CVoterank_selected, CV_capsP = CBVote(G, k)
print("CVoteRank done")

                             
######################################################################################################################

CV_capsP_sel = CVoterank_selected
                
######################################################################################################################
import pickle
try:  
   
    filep = open('CVoterank_selected', 'wb')
    pickle.dump(CVoterank_selected, filep)
    filep.close()
    
    print("Saved all pickale successfully")
    
except:
    print("Something went wrong")
    
    


2.25
CVoteRank done
Saved all pickale successfully


In [5]:
CVoterank_selected

[7, 15, 3, 9, 8]

In [ ]:
import math
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import random
import queue
import numpy as np
from operator import itemgetter
import csv


def SIRModel(g, spreaders, beta, S):
    infected_scale = np.array(np.zeros(T))
    ftc = np.array(np.zeros(T))
    while S > 0:
        S = S - 1
        infected = spreaders
        status = {}
        for i in g.nodes():
            status[i] = 0
        for i in infected:
            status[i] = 1
        n = g.number_of_nodes()
        infected_nodes = len(infected)
        recovered_nodes = 0
        time_stamp = 0
        infected_scale[time_stamp] = infected_scale[time_stamp] + (infected_nodes + recovered_nodes) / n
        infected = spreaders
        while len(infected) > 0:
            susceptible_to_infected = []
            time_stamp = time_stamp + 1
            for i in infected:
                susceptible = []
                status[i] = 2
                for neighbor in g.neighbors(i):
                    if status[neighbor] == 0:
                        susceptible.append(neighbor)
                total_susceptible = len(susceptible)
                no_of_susceptible_to_infected = round(beta * total_susceptible)
                while no_of_susceptible_to_infected > 0:
                    random_index = random.randint(0, total_susceptible - 1)
                    if susceptible[random_index] not in susceptible_to_infected:
                        susceptible_to_infected.append(susceptible[random_index])
                        status[susceptible[random_index]] = 1
                        no_of_susceptible_to_infected = no_of_susceptible_to_infected - 1
            infected_nodes = len(susceptible_to_infected)
            recovered_nodes = len(infected)
            ftc[time_stamp] = ftc[time_stamp] + recovered_nodes / n
            infected_scale[time_stamp] = infected_scale[time_stamp] + (infected_nodes + recovered_nodes) / n
            infected = susceptible_to_infected
    return infected_scale, ftc



def ftToftc(ft):
    ftc = [0 for i in range(0, T + 1)]
    for i in range(T):
        ftc[i + 1] = ft[i] + ftc[i]
    return ftc


def SPL(G, spreaders):
    total = 0
    n = len(spreaders)
    c = 0
    for i in range(0, n):
        node_i = spreaders[i]
        for j in range(i + 1, n):
            node_j = spreaders[j]
            if nx.has_path(G, spreaders[i], spreaders[j]):
                c = c + 1

                total = total + nx.shortest_path_length(G, node_i, node_j, 'weight')
    # print(total)
    # print(n)
    total = total / (n * n - n)
    total = 2 * total
    return total


def SIR(G, selected, beta, ft, ftc):
    Q = queue.Queue(maxsize=len(list(G.nodes())))
    recovered = []
    infected = []
    for node in selected:
        Q.put(node)
        infected.append(node)
    t = 0
    while t < T:
        tot = []
        while not Q.empty():
            curr = Q.get()
            if curr in recovered:
                continue
            recovered.append(curr)
            ne = list(G.neighbors(curr))
            random.shuffle(ne)
            neighbors = int(len(ne) * beta) + 1
            count = 0
            idx = 0
            while count < neighbors and idx < len(ne):
                if ne[idx] not in recovered and ne[idx] not in tot:
                    tot.append(ne[idx])
                    count += 1
                idx += 1
        for i in tot:
            Q.put(i)
        tot.clear()
        ftc[t] = ftc[t]+ len(recovered)
        ft[t] = ft[t]+len(recovered)+Q.qsize()
        t += 1
    return ft, ftc


def SIRcall(G, selected, beta, S):
    ft = [0.0 for i in range(T)]
    ftc = [0.0 for i in range(T)]
    for i in range(S):
        ft, ftc = SIR(G, selected, beta, ft, ftc)
    ft[:] = [x/S for x in ft]
    ftc[:] = [x/S for x in ftc]
    return ft, ftc


######################################################

##############################################
#for F(t) and F(tc) vs T experiment
k= int(0.05*(g.number_of_nodes()))
S=1000
T=50
beta=0.01
#print(k)
    
#From the ranking link, we select 0.05% spreaders 
CV_capsP_sel_p = CVoterank_selected[0:k]



ft_CV_capsP, ftc_CV_capsP = SIRcall(G, CV_capsP_sel_p, beta, S)
print("CV_capsP SIR Done")


######################################################################################################################
import pickle
try:  
   
    filep = open('CVoteRank_ft', 'wb')
    pickle.dump(ft_CV_capsP, filep)
    filep.close()
    
    filep = open('CVoteRank_ftc', 'wb')
    pickle.dump(ftc_CV_capsP, filep)
    filep.close()

    
    print("Saved all pickale successfully")
    
except:
    print("Something went wrong")
    

In [ ]:
ftc10 = [] #

# S = int(input("Input the number of times SIR should average out: "))
# beta = float(input("Input another beta: "))
S = 1000
beta = 0.01



spreaders_fractions_list = [0.03, 0.06, 0.09, 0.12, 0.15]
for fraction in spreaders_fractions_list:
    #print(fraction)
    spreaders = int(len(G) * fraction)
    #print(spreaders)
    
    # Proposed
    spreaders_list_CV_capsP = Find_Spreaders_Updated(G, CV_capsP_sel, spreaders)
    ft, ftc = SIRcall(G, spreaders_list_CV_capsP, beta, S)
    ftc10.append(ftc[len(ftc)-1])
    
######################################################################################################################
try:  
   
    filep = open('CVoteRank_fractions_list', 'wb')
    pickle.dump(ftc10, filep)
    filep.close()

    
    print("Saved all pickale successfully")
    
except:
    print("Something went wrong")

In [ ]:
ftc10 = [] #

S = 1000

beta_list = [0.01, 0.05, 0.10, 0.15]
for fraction in beta_list:
    print(fraction)
    spreaders=int(0.05*(g.number_of_nodes()))
    beta = fraction

    # Proposed
    spreaders_list_CV_capsP = Find_Spreaders_Updated(G, CV_capsP_sel_p, spreaders)
    ft, ftc = SIRcall(G, spreaders_list_CV_capsP, beta, S)
    ftc10.append(ftc[len(ftc)-1])

######################################################################################################################
try:  
   
    filep = open('CVoteRank_beta', 'wb')
    pickle.dump(ftc10, filep)
    filep.close()

    
    print("Saved all pickale successfully")
    
except:
    print("Something went wrong")


In [ ]:
def get_ls(g, infeacted_set):
    """compute the average shortest path in the initial node set
     # Arguments
         g: a graph as networkx Graph
         infeacted_set: the initial node set
     Returns
         return the average shortest path
     """
    dis_sum = 0
    path_num = 0
    for u in infeacted_set:
        for v in infeacted_set:
            if u != v:
                try:
                    dis_sum += nx.shortest_path_length(g, u, v)
                    path_num += 1
                except:
                    dis_sum += 0
                    path_num -= 1
    return dis_sum / path_num



from tqdm import tqdm
#r = [0.05, 0.1, 0.15, 0.2, 0.25]
r=[0.03,0.06,0.09,0.12,0.15]

topk_list = []
for k in r:
    topk = round(nx.number_of_nodes(G) * k)
    print(k, topk)
    topk_list.append(topk)
    
max_ = r[-1]
max_topk = round(max_ * nx.number_of_nodes(G))
print(max_topk)

  
CV_capsPRank = CV_capsP_sel


#######################################################################################################

CV_capsP_ls = []
#######################################################################################################    
for k in tqdm(topk_list):
    topk = k       
    CV_capsP_ls.append(get_ls(G, [x for x in CV_capsPRank[:topk]]))
    
####################################################################################################### 

try:  
   
    filep = open('CVoteRank_distance', 'wb')
    pickle.dump(CV_capsP_ls, filep)
    filep.close()

    
    print("Saved all pickale successfully")
    
except:
    print("Something went wrong")
